In [17]:
import stripe
import pandas as pd
import numpy as np
from time import sleep
from config import api_key
from config import host, database, user, password
from queries import insert_table
import psycopg2


ImportError: cannot import name 'insert_table' from 'queries' (C:\Users\John Stauffer\Documents\stripe-etl\queries.py)

In [2]:
def get_df():
    sub_df = pd.DataFrame()
    stripe.api_key = api_key
                                                        
    #first query to get response id
    first_response = stripe.Subscription.list(status='all')
    #creating dataframe with responses
    sub_df = sub_df.append(pd.json_normalize(first_response['data']))
    #defining next page for loops
    start_after = first_response['data'][-1]['id']
    print('Getting Results')

    while True:    
        try:
            #appending to the dataframe
            response = stripe.Subscription.list(starting_after=start_after, status='all')
            subscription_df = pd.json_normalize(response['data'])
            sub_df = sub_df.append(subscription_df, ignore_index=True)
            #giving the key for the next iteration
            start_after = response['data'][-1]['id']
            #ask george about rate limit issues
            sleep(1)
        except:
            print('Stripe Request went wrong')
        
        
        if response["has_more"] == False:
            break
    return sub_df


In [3]:
sub_df = get_df()

Getting Results


In [4]:
def cleaning_df(df):
    df = df[['id', 'billing_cycle_anchor', 'cancel_at', 'cancel_at_period_end', 'canceled_at', 'created', 'current_period_end', 'current_period_start', 'customer', 'ended_at', 'livemode', 'quantity', 'schedule', 'start_date', 'status']]
    df['billing_cycle_anchor'] = pd.to_datetime(sub_df['billing_cycle_anchor'], unit='s')
    df['canceled_at'] = pd.to_datetime(sub_df['canceled_at'], unit='s')
    df['created'] = pd.to_datetime(sub_df['created'], unit='s')
    df['current_period_end'] = pd.to_datetime(sub_df['current_period_end'], unit='s')
    df['current_period_start'] = pd.to_datetime(sub_df['current_period_start'], unit='s')
    df['ended_at'] = pd.to_datetime(sub_df['ended_at'], unit='s')
    df['start_date'] = pd.to_datetime(sub_df['start_date'], unit='s')
    return df


In [5]:
sub_df = cleaning_df(sub_df)

C:\Users\JOHNST~1\AppData\Local\Temp/ipykernel_28928/179360464.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['billing_cycle_anchor'] = pd.to_datetime(sub_df['billing_cycle_anchor'], unit='s')
C:\Users\JOHNST~1\AppData\Local\Temp/ipykernel_28928/179360464.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['canceled_at'] = pd.to_datetime(sub_df['canceled_at'], unit='s')
C:\Users\JOHNST~1\AppData\Local\Temp/ipykernel_28928/179360464.py:5: SettingWithCopyWarning: 
A value is trying to be set on a c

In [6]:
def connect(host, database, user, password):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(host=host, database=database, user=user, password=password)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn
conn = connect(host, database, user, password)

Connecting to the PostgreSQL database...
Connection successful


In [7]:
def setup(conn, schema, table):
    cur = conn.cursor()
    cur.execute(schema)
    cur.execute(table)
    

In [8]:
setup(conn, create_subscription_schema, create_subscription_table)

In [9]:
def insert_values(conn, df, table):
    
    # Create a list of tupples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL quert to execute
    cur = conn.cursor()
    values = [cur.mogrify("(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)", tup).decode('utf8') for tup in tuples]
    query  = (f'INSERT INTO {table(cols)} VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s) ON CONFLICT({values[0]}) DO UPDATE SET')
    
    try:
         cur.execute(query, tuples)
         conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cur.close()
        return 1
    print("execute_mogrify() done")
    cursor.close()
   

In [10]:
insert_values(conn, sub_df, 'subscriptions')

IndexError: tuple index out of range

In [ ]:
sub_df.columns

In [13]:
def insert_values_v2(conn, df):
    
    cur = conn.cursor()
    
    for value in df.values:
        subscription_id, billing_cycle_anchor, cancel_at, cancel_at_period_end, canceled_at, created, current_period_end, current_period_start, customer, ended_at, livemode, quantity, schedule, start_date, status_year = value

        # insert subscriber record
        subscriber_data = (subscription_id, billing_cycle_anchor, cancel_at, cancel_at_period_end, canceled_at, created, current_period_end, current_period_start, customer, ended_at, livemode, quantity, schedule, start_date, status_year)
        cur.execute(insert_table, subscriber_data)



In [16]:
insert_values_v2(conn, sub_df)

NameError: name 'insert_table' is not defined